In [ ]:
# Copyright 2021 NVIDIA CORPORATION
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
import os
import time
#os.environ["CUDA_VISIBLE_DEVICES"]="2,3"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import gc
import glob
import cupy as cp
import pandas as pd
from tqdm.notebook import tqdm
from joblib import Parallel, delayed
import os, time
from tqdm.notebook import tqdm
import pickle

import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import cudf, cupy, time, rmm
print( cudf.__version__ )

from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
from dask.utils import parse_bytes
import cudf
import dask_cudf

import os
import time
import glob
import gc

import cupy as cp          # CuPy is an implementation of NumPy-compatible multi-dimensional array on GPU
import cudf                # cuDF is an implementation of Pandas-like Dataframe on GPU
import rmm                 # library for pre-allocating memory on GPU
import dask                # dask is an open-source library to nateively scale Python on multiple workers/nodes
import dask_cudf           # dask_cudf uses dask to scale cuDF dataframes on multiple workers/nodes

import numpy as np
import time
import xgboost as xgb
print( 'xgb', xgb.__version__ )

# More dask / dask_cluster related libraries to scale NVTabular
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
from dask.distributed import wait
from dask.utils import parse_bytes
from dask.delayed import delayed

# NVTabular is the core library, we will use here for feature engineering/preprocessing on GPU
#import nvtabular as nvt
#from nvtabular.utils import device_mem_size
!rm -r dask-worker-space

import xgboost as xgb
print('XGB Version',xgb.__version__)

start = time.time()

0.18.0
xgb 1.3.3
rm: cannot remove 'dask-worker-space': No such file or directory
XGB Version 1.3.3


In [2]:
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import log_loss

def compute_rce_fast(pred, gt):
    cross_entropy = log_loss(gt, pred)
    yt = np.mean(gt)     
    strawman_cross_entropy = -(yt*np.log(yt) + (1 - yt)*np.log(1 - yt))
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

In [3]:
def convert_int32(df, feats=[]):
    for f in feats:
        df[f] = df[f].astype(np.int32)
    gc.collect()
    
def convert_int16(df, feats=[]):
    for f in feats:
        df[f] = df[f].astype(np.int16)
    gc.collect()
    
def convert_int8(df, feats=[]):
    for f in feats:
        df[f] = df[f].astype(np.int8)        
    gc.collect()

In [4]:
trainfiles = glob.glob('week3-valid/*.parquet' )
len(trainfiles), trainfiles

(253,
 ['week3-valid/part.44.parquet',
  'week3-valid/part.233.parquet',
  'week3-valid/part.119.parquet',
  'week3-valid/part.176.parquet',
  'week3-valid/part.198.parquet',
  'week3-valid/part.159.parquet',
  'week3-valid/part.60.parquet',
  'week3-valid/part.219.parquet',
  'week3-valid/part.69.parquet',
  'week3-valid/part.224.parquet',
  'week3-valid/part.20.parquet',
  'week3-valid/part.118.parquet',
  'week3-valid/part.154.parquet',
  'week3-valid/part.6.parquet',
  'week3-valid/part.54.parquet',
  'week3-valid/part.169.parquet',
  'week3-valid/part.65.parquet',
  'week3-valid/part.151.parquet',
  'week3-valid/part.79.parquet',
  'week3-valid/part.37.parquet',
  'week3-valid/part.197.parquet',
  'week3-valid/part.172.parquet',
  'week3-valid/part.41.parquet',
  'week3-valid/part.81.parquet',
  'week3-valid/part.155.parquet',
  'week3-valid/part.62.parquet',
  'week3-valid/part.82.parquet',
  'week3-valid/part.134.parquet',
  'week3-valid/part.143.parquet',
  'week3-valid/part.16

In [5]:
pd.read_parquet(trainfiles[0]).columns

Index(['hashtags', 'tweet_id', 'media', 'links', 'domains', 'tweet_type',
       'language', 'timestamp', 'a_user_id', 'a_follower_count',
       'a_following_count', 'a_is_verified', 'a_account_creation', 'b_user_id',
       'b_follower_count', 'b_following_count', 'b_is_verified',
       'b_account_creation', 'b_follows_a', 'reply', 'retweet',
       'retweet_comment', 'like', 'tw_len_token', 'tw_len_media',
       'tw_len_photo', 'tw_len_video', 'tw_len_gif', 'tw_len_quest',
       'tw_count_capital_words', 'tw_count_excl_quest_marks',
       'tw_count_special1', 'tw_count_hash', 'tw_last_quest', 'tw_len_retweet',
       'tw_len_rt', 'tw_count_at', 'tw_count_words', 'tw_count_char',
       'tw_rt_count_words', 'tw_rt_count_char', 'tw_original_user0',
       'tw_original_user1', 'tw_original_user2', 'tw_rt_user0', 'tw_word0',
       'tw_word1', 'tw_tweet', 'group', 'dt_day', 'dt_dow', 'dt_minute',
       'len_hashtags', 'len_links', 'len_domains', 'decline'],
      dtype='object')

In [6]:
%%time

train = pd.concat([pd.read_parquet(fn) for fn in trainfiles]).reset_index(drop=True)
train['decline'] = train['decline']//7.19
train['a_follower_count'] = train['a_follower_count'] // 819
train['a_following_count'] = train['a_following_count'] // 712
train['b_follower_count'] = train['b_follower_count'] // 819
train['b_following_count'] = train['b_following_count'] // 712
gc.collect()
train.shape

CPU times: user 35.6 s, sys: 24.3 s, total: 59.8 s
Wall time: 47.3 s


(47815171, 56)

In [7]:
train.columns

Index(['hashtags', 'tweet_id', 'media', 'links', 'domains', 'tweet_type',
       'language', 'timestamp', 'a_user_id', 'a_follower_count',
       'a_following_count', 'a_is_verified', 'a_account_creation', 'b_user_id',
       'b_follower_count', 'b_following_count', 'b_is_verified',
       'b_account_creation', 'b_follows_a', 'reply', 'retweet',
       'retweet_comment', 'like', 'tw_len_token', 'tw_len_media',
       'tw_len_photo', 'tw_len_video', 'tw_len_gif', 'tw_len_quest',
       'tw_count_capital_words', 'tw_count_excl_quest_marks',
       'tw_count_special1', 'tw_count_hash', 'tw_last_quest', 'tw_len_retweet',
       'tw_len_rt', 'tw_count_at', 'tw_count_words', 'tw_count_char',
       'tw_rt_count_words', 'tw_rt_count_char', 'tw_original_user0',
       'tw_original_user1', 'tw_original_user2', 'tw_rt_user0', 'tw_word0',
       'tw_word1', 'tw_tweet', 'group', 'dt_day', 'dt_dow', 'dt_minute',
       'len_hashtags', 'len_links', 'len_domains', 'decline'],
      dtype='object')

In [8]:
%%time

dt = pd.read_parquet('TEMAPS_LOCAL/a_user_id-tweet_type.parquet')
features = list(dt.index.names)
train = train.merge(dt, left_on=features, right_index=True, how='left')
del dt; gc.collect()
print(train.shape)

dt = pd.read_parquet('TEMAPS_LOCAL/b_user_id-tweet_type.parquet')
features = list(dt.index.names)
train = train.merge(dt, left_on=features, right_index=True, how='left')
del dt; gc.collect()
print(train.shape)

dt = pd.read_parquet('TEMAPS_LOCAL/media-language-tweet_type-a_is_verified-b_is_verified-b_follows_a-tw_last_quest-decline-group.parquet')
features = list(dt.index.names)
train = train.merge(dt, left_on=features, right_index=True, how='left')
del dt; gc.collect()
print(train.shape)

dt = pd.read_parquet('TEMAPS_LOCAL/tw_original_user0-tweet_type.parquet')
features = list(dt.index.names)
train = train.merge(dt, left_on=features, right_index=True, how='left')
del dt; gc.collect()
print(train.shape)

dt = pd.read_parquet('TEMAPS_LOCAL/tw_original_user1-tweet_type.parquet')
features = list(dt.index.names)
train = train.merge(dt, left_on=features, right_index=True, how='left')
del dt; gc.collect()
print(train.shape)

dt = pd.read_parquet('TEMAPS_LOCAL/tw_word0-tweet_type.parquet')
features = list(dt.index.names)
train = train.merge(dt, left_on=features, right_index=True, how='left')
del dt; gc.collect()
print(train.shape)

dt = pd.read_parquet('TEMAPS_LOCAL/tw_rt_user0.parquet')
features = list(dt.index.names)
train = train.merge(dt, left_on=features, right_index=True, how='left')
del dt; gc.collect()
print(train.shape)

dt = pd.read_parquet('TEMAPS_LOCAL/a_follower_count-a_following_count-b_follower_count-b_following_count-tweet_type-language-b_follows_a.parquet')
features = list(dt.index.names)
train = train.merge(dt, left_on=features, right_index=True, how='left')
del dt; gc.collect()
print(train.shape)

dt = pd.read_parquet('TEMAPS_LOCAL/b_user_id.parquet')
features = list(dt.index.names)
train = train.merge(dt, left_on=features, right_index=True, how='left')
del dt; gc.collect()
print(train.shape)

(47815171, 61)
(47815171, 66)
(47815171, 71)
(47815171, 76)
(47815171, 81)
(47815171, 86)
(47815171, 91)
(47815171, 96)
(47815171, 101)
CPU times: user 3min 41s, sys: 45.9 s, total: 4min 27s
Wall time: 4min 24s


In [9]:
%%time

valid = pd.read_parquet('/raid/kaggle/2021/recsys/input/validation.parquet')
valid['decline'] = valid['decline']//7.19
valid['a_follower_count'] = valid['a_follower_count'] // 819
valid['a_following_count'] = valid['a_following_count'] // 712
valid['b_follower_count'] = valid['b_follower_count'] // 819
valid['b_following_count'] = valid['b_following_count'] // 712
gc.collect()
valid.shape

CPU times: user 19 s, sys: 7.69 s, total: 26.7 s
Wall time: 22.3 s


(14461760, 61)

In [10]:
%%time

dt = pd.read_parquet('TEMAPS_LOCAL/a_user_id-tweet_type.parquet')
features = list(dt.index.names)
valid = valid.merge(dt, left_on=features, right_index=True, how='left')
del dt; gc.collect()
print(valid.shape)

dt = pd.read_parquet('TEMAPS_LOCAL/b_user_id-tweet_type.parquet')
features = list(dt.index.names)
valid = valid.merge(dt, left_on=features, right_index=True, how='left')
del dt; gc.collect()
print(valid.shape)

dt = pd.read_parquet('TEMAPS_LOCAL/media-language-tweet_type-a_is_verified-b_is_verified-b_follows_a-tw_last_quest-decline-group.parquet')
features = list(dt.index.names)
valid = valid.merge(dt, left_on=features, right_index=True, how='left')
del dt; gc.collect()
print(valid.shape)

dt = pd.read_parquet('TEMAPS_LOCAL/tw_original_user0-tweet_type.parquet')
features = list(dt.index.names)
valid = valid.merge(dt, left_on=features, right_index=True, how='left')
del dt; gc.collect()
print(valid.shape)

dt = pd.read_parquet('TEMAPS_LOCAL/tw_original_user1-tweet_type.parquet')
features = list(dt.index.names)
valid = valid.merge(dt, left_on=features, right_index=True, how='left')
del dt; gc.collect()
print(valid.shape)

dt = pd.read_parquet('TEMAPS_LOCAL/tw_word0-tweet_type.parquet')
features = list(dt.index.names)
valid = valid.merge(dt, left_on=features, right_index=True, how='left')
del dt; gc.collect()
print(valid.shape)

dt = pd.read_parquet('TEMAPS_LOCAL/tw_rt_user0.parquet')
features = list(dt.index.names)
valid = valid.merge(dt, left_on=features, right_index=True, how='left')
del dt; gc.collect()
print(valid.shape)

dt = pd.read_parquet('TEMAPS_LOCAL/a_follower_count-a_following_count-b_follower_count-b_following_count-tweet_type-language-b_follows_a.parquet')
features = list(dt.index.names)
valid = valid.merge(dt, left_on=features, right_index=True, how='left')
del dt; gc.collect()
print(valid.shape)

dt = pd.read_parquet('TEMAPS_LOCAL/b_user_id.parquet')
features = list(dt.index.names)
valid = valid.merge(dt, left_on=features, right_index=True, how='left')
del dt; gc.collect()
print(valid.shape)

(14461760, 66)
(14461760, 71)
(14461760, 76)
(14461760, 81)
(14461760, 86)
(14461760, 91)
(14461760, 96)
(14461760, 101)
(14461760, 106)
CPU times: user 2min 18s, sys: 29.7 s, total: 2min 48s
Wall time: 2min 43s


In [12]:
for f in [
'sums_te_reply-a_user_id-tweet_type',
       'counts_te_reply-a_user_id-tweet_type',
       'sums_te_retweet-a_user_id-tweet_type',
       'sums_te_retweet_comment-a_user_id-tweet_type',
       'sums_te_like-a_user_id-tweet_type',
       'sums_te_reply-b_user_id-tweet_type',
       'counts_te_reply-b_user_id-tweet_type',
       'sums_te_retweet-b_user_id-tweet_type',
       'sums_te_retweet_comment-b_user_id-tweet_type',
       'sums_te_like-b_user_id-tweet_type', 'multi_reply', 'multi_retweet',
       'multi_retweet_comment', 'multi_like', 'multi_counts', 'ouser0_reply',
       'ouser0_retweet', 'ouser0_retweet_comment', 'ouser0_like',
       'ouser0_counts', 'ouser1_reply', 'ouser1_retweet',
       'ouser1_retweet_comment', 'ouser1_like', 'ouser1_counts', 'word_reply',
       'word_retweet', 'word_retweet_comment', 'word_like', 'word_counts',
       'rtuser0_reply', 'rtuser0_retweet', 'rtuser0_retweet_comment',
       'rtuser0_like', 'rtuser0_counts', 'follow_reply', 'follow_retweet',
       'follow_retweet_comment', 'follow_like', 'follow_counts',
       'sums_b_user_reply', 'sums_b_user_retweet',
       'sums_b_user_retweet_comment', 'sums_b_user_like', 'counts_b_user_id'      
]:
    train[f] = train[f].astype(np.float32)
    valid[f] = valid[f].astype(np.float32)
gc.collect()

0

In [13]:
train.head()

,hashtags,tweet_id,media,links,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,...,follow_reply,follow_retweet,follow_retweet_comment,follow_like,follow_counts,sums_b_user_reply,sums_b_user_retweet,sums_b_user_retweet_comment,sums_b_user_like,counts_b_user_id
0,0,-2872051519089231643,0,0,0,0,0,1613609666,3264529006259093641,1882,...,0.0,0.0,0.0,0.0,41.0,1.0,0.0,0.0,4.0,14.0
1,0,5452922862876665862,0,0,0,0,0,1613939869,958129457003958467,145,...,445.0,738.0,97.0,6021.0,8905.0,0.0,1.0,0.0,1.0,10.0
2,0,4871374771923015139,0,0,0,1,3,1613871462,-6936296245275888793,4,...,41.0,427.0,46.0,1533.0,4538.0,0.0,6.0,0.0,12.0,40.0
3,0,5187395191713456404,0,2031127319,495726603,1,13,1614152397,5433276069498045353,43,...,4.0,49.0,0.0,115.0,667.0,0.0,1.0,0.0,39.0,69.0
4,138641100,815427420061189949,1,0,0,0,0,1613940199,309709089998636133,0,...,8082.0,7714.0,710.0,69799.0,150246.0,0.0,0.0,0.0,3.0,6.0


In [39]:
targets = ['reply', 'retweet', 'retweet_comment', 'like']

features_base = [
    'media', 'tweet_type', 'language', 
    'a_follower_count', 'a_following_count', 'a_is_verified', 'a_account_creation',
    'b_follower_count', 'b_following_count', 'b_is_verified', 'b_account_creation', 'b_follows_a',
    'tw_len_token', 'tw_len_media', 'tw_len_photo', 'tw_len_video', 'tw_len_gif',
    'tw_len_quest', 'tw_count_capital_words', 'tw_count_excl_quest_marks', 'tw_count_special1',
    'tw_count_hash', 'tw_last_quest', 'tw_len_retweet', 'tw_len_rt', 'tw_count_at',
    'tw_count_words', 'tw_count_char',  'tw_rt_count_words', 'tw_rt_count_char',
    #'group',
    'dt_dow',
    #'dt_minute',
    'len_hashtags', 'len_links', 'len_domains', 'decline'
]

In [44]:
feature_reply = [
    
'sums_te_reply-a_user_id-tweet_type',
#'sums_te_retweet-a_user_id-tweet_type',
#'sums_te_retweet_comment-a_user_id-tweet_type',
#'sums_te_like-a_user_id-tweet_type',
'counts_te_reply-a_user_id-tweet_type',
    
'sums_te_reply-b_user_id-tweet_type',
#'sums_te_retweet-b_user_id-tweet_type',
#'sums_te_retweet_comment-b_user_id-tweet_type',
#'sums_te_like-b_user_id-tweet_type',
'counts_te_reply-b_user_id-tweet_type',
    
'multi_reply',
'multi_retweet',
'multi_retweet_comment',
'multi_like',
'multi_counts', 

'ouser0_reply',
'ouser0_retweet',
'ouser0_retweet_comment', 
'ouser0_like',
'ouser0_counts',

'ouser1_reply',
#'ouser1_retweet',
#'ouser1_retweet_comment', 
#'ouser1_like',
'ouser1_counts',

'word_reply',
'word_retweet',
'word_retweet_comment',
'word_like',
'word_counts',

'rtuser0_reply',
#'rtuser0_retweet',
#'rtuser0_retweet_comment',
#'rtuser0_like',
'rtuser0_counts',
    
'follow_reply',
'follow_retweet',
'follow_retweet_comment',
'follow_like',
'follow_counts',
    
'sums_b_user_reply',
'sums_b_user_retweet',
'sums_b_user_retweet_comment',
'sums_b_user_like',
'counts_b_user_id',
    
]

feature_reply

['sums_te_reply-a_user_id-tweet_type',
 'counts_te_reply-a_user_id-tweet_type',
 'sums_te_reply-b_user_id-tweet_type',
 'counts_te_reply-b_user_id-tweet_type',
 'multi_reply',
 'multi_retweet',
 'multi_retweet_comment',
 'multi_like',
 'multi_counts',
 'ouser0_reply',
 'ouser0_retweet',
 'ouser0_retweet_comment',
 'ouser0_like',
 'ouser0_counts',
 'ouser1_reply',
 'ouser1_retweet',
 'ouser1_retweet_comment',
 'ouser1_like',
 'ouser1_counts',
 'word_reply',
 'word_retweet',
 'word_retweet_comment',
 'word_like',
 'word_counts',
 'rtuser0_reply',
 'rtuser0_retweet',
 'rtuser0_retweet_comment',
 'rtuser0_like',
 'rtuser0_counts',
 'follow_reply',
 'follow_retweet',
 'follow_retweet_comment',
 'follow_like',
 'follow_counts',
 'sums_b_user_reply',
 'sums_b_user_retweet',
 'sums_b_user_retweet_comment',
 'sums_b_user_like',
 'counts_b_user_id']

In [45]:
features = features_base+feature_reply
pickle.dump(features, open('xgbmodels-features-reply.pickle', 'wb'))
features

['media',
 'tweet_type',
 'language',
 'a_follower_count',
 'a_following_count',
 'a_is_verified',
 'a_account_creation',
 'b_follower_count',
 'b_following_count',
 'b_is_verified',
 'b_account_creation',
 'b_follows_a',
 'tw_len_token',
 'tw_len_media',
 'tw_len_photo',
 'tw_len_video',
 'tw_len_gif',
 'tw_len_quest',
 'tw_count_capital_words',
 'tw_count_excl_quest_marks',
 'tw_count_special1',
 'tw_count_hash',
 'tw_last_quest',
 'tw_len_retweet',
 'tw_len_rt',
 'tw_count_at',
 'tw_count_words',
 'tw_count_char',
 'tw_rt_count_words',
 'tw_rt_count_char',
 'dt_dow',
 'len_hashtags',
 'len_links',
 'len_domains',
 'decline',
 'sums_te_reply-a_user_id-tweet_type',
 'counts_te_reply-a_user_id-tweet_type',
 'sums_te_reply-b_user_id-tweet_type',
 'counts_te_reply-b_user_id-tweet_type',
 'multi_reply',
 'multi_retweet',
 'multi_retweet_comment',
 'multi_like',
 'multi_counts',
 'ouser0_reply',
 'ouser0_retweet',
 'ouser0_retweet_comment',
 'ouser0_like',
 'ouser0_counts',
 'ouser1_reply'

In [46]:
train[features].head()

,media,tweet_type,language,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_follower_count,b_following_count,b_is_verified,...,follow_reply,follow_retweet,follow_retweet_comment,follow_like,follow_counts,sums_b_user_reply,sums_b_user_retweet,sums_b_user_retweet_comment,sums_b_user_like,counts_b_user_id
0,0,0,0,1882,808,1,15,0,2,0,...,0.0,0.0,0.0,0.0,41.0,1.0,0.0,0.0,4.0,14.0
1,0,0,0,145,4,1,-64,0,0,0,...,445.0,738.0,97.0,6021.0,8905.0,0.0,1.0,0.0,1.0,10.0
2,0,1,3,4,6,0,-123,0,1,0,...,41.0,427.0,46.0,1533.0,4538.0,0.0,6.0,0.0,12.0,40.0
3,0,1,13,43,0,0,-38,0,0,0,...,4.0,49.0,0.0,115.0,667.0,0.0,1.0,0.0,39.0,69.0
4,1,0,0,0,1,0,-71,0,0,0,...,8082.0,7714.0,710.0,69799.0,150246.0,0.0,0.0,0.0,3.0,6.0


In [25]:
train['fold'] = np.arange(train.shape[0]) % 5  #TRAIN['tweet_id'] % 5
print(train.groupby('fold')['media'].agg('count'))
gc.collect();gc.collect();gc.collect();

fold
0    9563035
1    9563034
2    9563034
3    9563034
4    9563034
Name: media, dtype: int64


In [26]:
train['positive'] = 0
train.loc[(train['reply']>0)|(train['retweet']>0)|(train['retweet_comment']>0)|(train['like']>0), 'positive'] = 1
print(train['positive'].value_counts())

train['target'] = 0
train.loc[train['positive']==1,'target'] = 1
train.loc[train['reply']==1,'target'] = 2
print( train['target'].value_counts() )

0    24114352
1    23700819
Name: positive, dtype: int64
1    7323312
0    7138448
Name: positive, dtype: int64
0    24114352
1    22337228
2     1363591
Name: target, dtype: int64
0    7138448
1    7018751
2     304561
Name: target, dtype: int64


In [65]:
#!rm -r giba_xgbmodels
#!mkdir giba_xgbmodels
xgb_parms = { 
    'max_depth': 8, 
    'learning_rate':0.04, 
    'subsample':0.50,
    'sampling_method': 'gradient_based',
    'colsample_bytree':0.80, 
    'grow_policy ': 'lossguide', #'depthwise', 'lossguide'
    'eval_metric':'auc',
    'objective':'binary:logistic',
    'nthread': 1,
    'tree_method':'gpu_hist',
    'predictor' : 'gpu_predictor'
}
print(xgb_parms)

target = 'reply'

feature_reply = [
    
'sums_te_reply-a_user_id-tweet_type',
#'sums_te_retweet-a_user_id-tweet_type',
#'sums_te_retweet_comment-a_user_id-tweet_type',
#'sums_te_like-a_user_id-tweet_type',
'counts_te_reply-a_user_id-tweet_type',
    
'sums_te_reply-b_user_id-tweet_type',
#'sums_te_retweet-b_user_id-tweet_type',
#'sums_te_retweet_comment-b_user_id-tweet_type',
#'sums_te_like-b_user_id-tweet_type',
'counts_te_reply-b_user_id-tweet_type',
    
'multi_reply',
'multi_retweet',
'multi_retweet_comment',
'multi_like',
'multi_counts', 

'ouser0_reply',
'ouser0_retweet',
'ouser0_retweet_comment', 
'ouser0_like',
'ouser0_counts',

'ouser1_reply',
#'ouser1_retweet',
#'ouser1_retweet_comment', 
#'ouser1_like',
'ouser1_counts',

'word_reply',
'word_retweet',
'word_retweet_comment',
'word_like',
'word_counts',

'rtuser0_reply',
#'rtuser0_retweet',
#'rtuser0_retweet_comment',
#'rtuser0_like',
'rtuser0_counts',
    
'follow_reply',
'follow_retweet',
'follow_retweet_comment',
'follow_like',
'follow_counts',
    
'sums_b_user_reply',
'sums_b_user_retweet',
'sums_b_user_retweet_comment',
'sums_b_user_like',
'counts_b_user_id',
    
]
features = features_base+feature_reply
pickle.dump(features, open('xgbmodels-features-reply.pickle', 'wb'))

print('TARGET:', target )
for group in range(5):
    start = time.time();
    dtrain = xgb.DMatrix(data=train.loc[(train.group==group), features], label=train.loc[(train.group==group), target] ) )
    gc.collect();gc.collect();gc.collect();
    print( time.time() - start, 's' )

    start = time.time(); print('Training...')
    model = xgb.train(
        xgb_parms, 
        dtrain=dtrain,
        #evals=[(dvalid2,'valid2')],
        num_boost_round=550,
        #early_stopping_rounds=100,
        verbose_eval=100,
    )
    pickle.dump(model, open('giba_xgbmodels/model_' + str(target) + '_' + str(group) + '.pickle', 'wb'))


{'max_depth': 8, 'learning_rate': 0.04, 'subsample': 0.5, 'sampling_method': 'gradient_based', 'colsample_bytree': 0.8, 'grow_policy ': 'lossguide', 'eval_metric': 'auc', 'objective': 'binary:logistic', 'nthread': 1, 'tree_method': 'gpu_hist', 'predictor': 'gpu_predictor'}
TARGET: reply
10.439348697662354 s
Training...
[18:20:38] WARNING: /opt/conda/envs/rapids/conda-bld/xgboost_1614189954438/work/src/learner.cc:541: 
Parameters: { grow_policy  } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


549 RCE: 0 20.810878816579226 0 0.20378615952476595 32 s

10.280435800552368 s
Training...
[18:21:22] WARNING: /opt/conda/envs/rapids/conda-bld/xgboost_1614189954438/work/src/learner.cc:541: 
Parameters: { grow_policy  } might not be used.

  This may not be accurate due to some parameters

In [61]:
xgb_parms = { 
    'max_depth': 8, 
    'learning_rate':0.04, 
    'subsample':0.50,
    'sampling_method': 'gradient_based',
    'colsample_bytree':0.80, 
    'grow_policy ': 'lossguide', #'depthwise', 'lossguide'
    'eval_metric':'auc',
    'objective':'binary:logistic',
    'nthread': 1,
    'tree_method':'gpu_hist',
    'predictor' : 'gpu_predictor'
}

target = 'retweet'

feature_retweet = [
    
#'sums_te_reply-a_user_id-tweet_type',
'sums_te_retweet-a_user_id-tweet_type',
#'sums_te_retweet_comment-a_user_id-tweet_type',
#'sums_te_like-a_user_id-tweet_type',
'counts_te_reply-a_user_id-tweet_type',
    
#'sums_te_reply-b_user_id-tweet_type',
'sums_te_retweet-b_user_id-tweet_type',
#'sums_te_retweet_comment-b_user_id-tweet_type',
#'sums_te_like-b_user_id-tweet_type',
'counts_te_reply-b_user_id-tweet_type',
    
'multi_reply',
'multi_retweet',
'multi_retweet_comment',
'multi_like',
'multi_counts', 

'ouser0_reply',
'ouser0_retweet',
'ouser0_retweet_comment', 
'ouser0_like',
'ouser0_counts',

#'ouser1_reply',
'ouser1_retweet',
#'ouser1_retweet_comment', 
#'ouser1_like',
'ouser1_counts',

'word_reply',
'word_retweet',
'word_retweet_comment',
'word_like',
'word_counts',

#'rtuser0_reply',
'rtuser0_retweet',
#'rtuser0_retweet_comment',
#'rtuser0_like',
'rtuser0_counts',
    
'follow_reply',
'follow_retweet',
'follow_retweet_comment',
'follow_like',
'follow_counts',
    
'sums_b_user_reply',
'sums_b_user_retweet',
'sums_b_user_retweet_comment',
'sums_b_user_like',
'counts_b_user_id',
    
]
features = features_base+feature_retweet
pickle.dump(features, open('xgbmodels-features-retweet.pickle', 'wb'))

print('TARGET:', target )
for group in range(5):
    start = time.time();
    dtrain = xgb.DMatrix(data=train.loc[(train.group==group), features], label=train.loc[(train.group==group), target] ) )
    gc.collect();gc.collect();gc.collect();
    print( time.time() - start, 's' )

    start = time.time(); print('Training...')
    model = xgb.train(
        xgb_parms, 
        dtrain=dtrain,
        #evals=[(dvalid2,'valid2')],
        num_boost_round=615,
        #early_stopping_rounds=100,
        verbose_eval=100,
    )
    pickle.dump(model, open('giba_xgbmodels/model_' + str(target) + '_' + str(group) + '.pickle', 'wb'))


TARGET: retweet
9.789488077163696 s
Training...
[17:54:24] WARNING: /opt/conda/envs/rapids/conda-bld/xgboost_1614189954438/work/src/learner.cc:541: 
Parameters: { grow_policy  } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


614 RCE: 0 21.439275914227473 0 0.37303359671381164 36 s

10.148358583450317 s
Training...
[17:55:12] WARNING: /opt/conda/envs/rapids/conda-bld/xgboost_1614189954438/work/src/learner.cc:541: 
Parameters: { grow_policy  } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


614 RCE: 0 21.266120489837228 0 0.3596788321632822 92 s

11.381099700927734 s
Train

In [66]:
xgb_parms = { 
    'max_depth': 8, 
    'learning_rate':0.02, 
    'subsample':0.50,
    'sampling_method': 'gradient_based',
    'colsample_bytree':0.80, 
    'grow_policy ': 'lossguide', #'depthwise', 'lossguide'
    'eval_metric':'auc',
    'objective':'binary:logistic',
    'nthread': 1,
    'tree_method':'gpu_hist',
    'predictor' : 'gpu_predictor'
}

target = 'retweet_comment'

feature_retweet_comment = [
    
#'sums_te_reply-a_user_id-tweet_type',
#'sums_te_retweet-a_user_id-tweet_type',
'sums_te_retweet_comment-a_user_id-tweet_type',
#'sums_te_like-a_user_id-tweet_type',
'counts_te_reply-a_user_id-tweet_type',
    
#'sums_te_reply-b_user_id-tweet_type',
#'sums_te_retweet-b_user_id-tweet_type',
'sums_te_retweet_comment-b_user_id-tweet_type',
#'sums_te_like-b_user_id-tweet_type',
'counts_te_reply-b_user_id-tweet_type',
    
'multi_reply',
'multi_retweet',
'multi_retweet_comment',
'multi_like',
'multi_counts', 

'ouser0_reply',
'ouser0_retweet',
'ouser0_retweet_comment', 
'ouser0_like',
'ouser0_counts',

#'ouser1_reply',
#'ouser1_retweet',
'ouser1_retweet_comment', 
#'ouser1_like',
'ouser1_counts',

'word_reply',
'word_retweet',
'word_retweet_comment',
'word_like',
'word_counts',

#'rtuser0_reply',
#'rtuser0_retweet',
'rtuser0_retweet_comment',
#'rtuser0_like',
'rtuser0_counts',
    
'follow_reply',
'follow_retweet',
'follow_retweet_comment',
'follow_like',
'follow_counts',
    
'sums_b_user_reply',
'sums_b_user_retweet',
'sums_b_user_retweet_comment',
'sums_b_user_like',
'counts_b_user_id',
    
]
features = features_base+feature_retweet_comment
pickle.dump(features, open('xgbmodels-features-retweet_comment.pickle', 'wb'))

print('TARGET:', target )
for group in range(5):
    start = time.time();
    dtrain = xgb.DMatrix(data=train.loc[(train.group==group), features], label=train.loc[(train.group==group), target] ) )
    gc.collect();gc.collect();gc.collect();
    print( time.time() - start, 's' )

    start = time.time(); print('Training...')
    model = xgb.train(
        xgb_parms, 
        dtrain=dtrain,
        #evals=[(dvalid2,'valid2')],
        num_boost_round=800,
        #early_stopping_rounds=100,
        verbose_eval=100,
    )
    pickle.dump(model, open('giba_xgbmodels/model_' + str(target) + '_' + str(group) + '.pickle', 'wb'))


TARGET: retweet_comment
9.903303861618042 s
Training...
[18:35:43] WARNING: /opt/conda/envs/rapids/conda-bld/xgboost_1614189954438/work/src/learner.cc:541: 
Parameters: { grow_policy  } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


799 RCE: 0 12.932292869020445 0 0.05382582526608513 46 s

10.25521993637085 s
Training...
[18:36:41] WARNING: /opt/conda/envs/rapids/conda-bld/xgboost_1614189954438/work/src/learner.cc:541: 
Parameters: { grow_policy  } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


799 RCE: 0 11.963718880482688 0 0.04623983074268029 50 s

11.473490953445435

In [67]:
xgb_parms = { 
    'max_depth': 8, 
    'learning_rate':0.04, 
    'subsample':0.50,
    'sampling_method': 'gradient_based',
    'colsample_bytree':0.80, 
    'grow_policy ': 'lossguide', #'depthwise', 'lossguide'
    'eval_metric':'auc',
    'objective':'binary:logistic',
    'nthread': 1,
    'tree_method':'gpu_hist',
    'predictor' : 'gpu_predictor'
}

target = 'like'

feature_like = [
    
#'sums_te_reply-a_user_id-tweet_type',
#'sums_te_retweet-a_user_id-tweet_type',
#'sums_te_retweet_comment-a_user_id-tweet_type',
'sums_te_like-a_user_id-tweet_type',
'counts_te_reply-a_user_id-tweet_type',
    
#'sums_te_reply-b_user_id-tweet_type',
#'sums_te_retweet-b_user_id-tweet_type',
#'sums_te_retweet_comment-b_user_id-tweet_type',
'sums_te_like-b_user_id-tweet_type',
'counts_te_reply-b_user_id-tweet_type',
    
'multi_reply',
'multi_retweet',
'multi_retweet_comment',
'multi_like',
'multi_counts', 

'ouser0_reply',
'ouser0_retweet',
'ouser0_retweet_comment', 
'ouser0_like',
'ouser0_counts',

#'ouser1_reply',
#'ouser1_retweet',
#'ouser1_retweet_comment', 
'ouser1_like',
'ouser1_counts',

'word_reply',
'word_retweet',
'word_retweet_comment',
'word_like',
'word_counts',

#'rtuser0_reply',
#'rtuser0_retweet',
#'rtuser0_retweet_comment',
'rtuser0_like',
'rtuser0_counts',
    
'follow_reply',
'follow_retweet',
'follow_retweet_comment',
'follow_like',
'follow_counts',
    
'sums_b_user_reply',
'sums_b_user_retweet',
'sums_b_user_retweet_comment',
'sums_b_user_like',
'counts_b_user_id',
    
]
features = features_base+feature_like
pickle.dump(features, open('xgbmodels-features-like.pickle', 'wb'))

print('TARGET:', target )
for group in range(5):
    start = time.time();
    dtrain = xgb.DMatrix(data=train.loc[(train.group==group), features], label=train.loc[(train.group==group), target] ) )
    gc.collect();gc.collect();gc.collect();
    print( time.time() - start, 's' )

    start = time.time(); print('Training...')
    model = xgb.train(
        xgb_parms, 
        dtrain=dtrain,
        #evals=[(dvalid2,'valid2')],
        num_boost_round=694,
        #early_stopping_rounds=100,
        verbose_eval=100,
    )
    pickle.dump(model, open('giba_xgbmodels/model_' + str(target) + '_' + str(group) + '.pickle', 'wb'))


TARGET: like
9.91366696357727 s
Training...
[18:56:30] WARNING: /opt/conda/envs/rapids/conda-bld/xgboost_1614189954438/work/src/learner.cc:541: 
Parameters: { grow_policy  } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


693 RCE: 0 11.246404484904705 0 0.6075175594082944 40 s

10.262964010238647 s
Training...
[18:57:22] WARNING: /opt/conda/envs/rapids/conda-bld/xgboost_1614189954438/work/src/learner.cc:541: 
Parameters: { grow_policy  } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


693 RCE: 0 9.25918961254254 0 0.5904451382824741 98 s

11.477341413497925 s
Training...


In [68]:
!zip -r xgb.zip giba_xgbmodels/*

  adding: giba_xgbmodels/model_like_0.pickle (deflated 73%)
  adding: giba_xgbmodels/model_like_1.pickle (deflated 73%)
  adding: giba_xgbmodels/model_like_2.pickle (deflated 73%)
  adding: giba_xgbmodels/model_like_3.pickle (deflated 73%)
  adding: giba_xgbmodels/model_like_4.pickle (deflated 74%)
  adding: giba_xgbmodels/model_reply_0.pickle (deflated 72%)
  adding: giba_xgbmodels/model_reply_1.pickle (deflated 72%)
  adding: giba_xgbmodels/model_reply_2.pickle (deflated 73%)
  adding: giba_xgbmodels/model_reply_3.pickle (deflated 73%)
  adding: giba_xgbmodels/model_reply_4.pickle (deflated 73%)
  adding: giba_xgbmodels/model_retweet_0.pickle (deflated 73%)
  adding: giba_xgbmodels/model_retweet_1.pickle (deflated 73%)
  adding: giba_xgbmodels/model_retweet_2.pickle (deflated 73%)
  adding: giba_xgbmodels/model_retweet_3.pickle (deflated 73%)
  adding: giba_xgbmodels/model_retweet_4.pickle (deflated 74%)
  adding: giba_xgbmodels/model_retweet_comment_0.pickle (deflated 73%)
  adding:

In [69]:
pickle.load(open('xgbmodels-features-like.pickle', 'rb'))

['media',
 'tweet_type',
 'language',
 'a_follower_count',
 'a_following_count',
 'a_is_verified',
 'a_account_creation',
 'b_follower_count',
 'b_following_count',
 'b_is_verified',
 'b_account_creation',
 'b_follows_a',
 'tw_len_token',
 'tw_len_media',
 'tw_len_photo',
 'tw_len_video',
 'tw_len_gif',
 'tw_len_quest',
 'tw_count_capital_words',
 'tw_count_excl_quest_marks',
 'tw_count_special1',
 'tw_count_hash',
 'tw_last_quest',
 'tw_len_retweet',
 'tw_len_rt',
 'tw_count_at',
 'tw_count_words',
 'tw_count_char',
 'tw_rt_count_words',
 'tw_rt_count_char',
 'dt_dow',
 'len_hashtags',
 'len_links',
 'len_domains',
 'decline',
 'sums_te_like-a_user_id-tweet_type',
 'counts_te_reply-a_user_id-tweet_type',
 'sums_te_like-b_user_id-tweet_type',
 'counts_te_reply-b_user_id-tweet_type',
 'multi_reply',
 'multi_retweet',
 'multi_retweet_comment',
 'multi_like',
 'multi_counts',
 'ouser0_reply',
 'ouser0_retweet',
 'ouser0_retweet_comment',
 'ouser0_like',
 'ouser0_counts',
 'ouser1_like',
 